# 🔍 Fraud Detection with Logistic Regression from Scratch

## Tugas Besar 2 IF3070 – Dasar Inteligensi Artifisial

**Author:** AbyuDAIya-Ganbatte Team

This notebook implements a complete fraud detection pipeline using:
- **Logistic Regression** implemented from scratch with NumPy
- **Adam Optimizer** for efficient gradient descent
- **KNN Imputation** for handling missing values
- **Feature Engineering** with domain-specific transformations
- **Elastic Net Regularization** (L1 + L2) for preventing overfitting

---

## Table of Contents

1. [Import Libraries](#1-import-libraries)
2. [Load Data](#2-load-data)
3. [Data Exploration](#3-data-exploration)
4. [Preprocessing Pipeline](#4-preprocessing)
   - Separate Features and Target
   - KNN Imputation
   - Feature Engineering (Ratio, Log, Interaction Features)
   - One-Hot Encoding
   - Outlier Clipping
   - Standardization
5. [Logistic Regression Algorithm](#5-logistic-regression)
6. [Model Training & Evaluation](#6-training)
7. [Generate Submission](#7-submission)

## 1. Import Required Libraries

We use only `numpy` and `pandas` for the core implementation, with `KNNImputer` from sklearn for handling missing values.

In [14]:
import numpy as np
import pandas as pd
import json
from sklearn.impute import KNNImputer

# Set display options
pd.set_option('display.max_columns', None)
np.random.seed(42)

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Libraries imported successfully!
NumPy version: 2.3.3
Pandas version: 2.3.3


## 2. Load Training and Test Data

Load the fraud detection datasets from CSV files.

In [15]:
# Load datasets
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print("=" * 60)
print("DATA LOADED SUCCESSFULLY")
print("=" * 60)
print(f"\nTraining data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"\nTraining columns: {list(train_df.columns)}")
print(f"\nTest columns: {list(test_df.columns)}")

DATA LOADED SUCCESSFULLY

Training data shape: (100000, 31)
Test data shape: (100000, 30)

Training columns: ['ID', 'transaction_id', 'user_id', 'age', 'gender', 'country', 'device_type', 'device_os', 'merchant_category', 'transaction_amount', 'transaction_type', 'time_of_day', 'day_of_week', 'transaction_duration', 'num_prev_transactions', 'avg_transaction_amount', 'std_transaction_amount', 'transactions_last_24h', 'transactions_last_1h', 'failed_login_attempts', 'ip_risk_score', 'device_trust_score', 'account_age_days', 'has_chargeback_history', 'shared_ip_users', 'shared_device_users', 'merchant_risk', 'country_risk', 'distance_from_home', 'is_new_country', 'is_fraud']

Test columns: ['ID', 'transaction_id', 'user_id', 'age', 'gender', 'country', 'device_type', 'device_os', 'merchant_category', 'transaction_amount', 'transaction_type', 'time_of_day', 'day_of_week', 'transaction_duration', 'num_prev_transactions', 'avg_transaction_amount', 'std_transaction_amount', 'transactions_last

## 3. Initial Data Exploration

Explore the data to understand fraud rate, missing values, and data types.

In [16]:
# Check fraud rate (target distribution)
fraud_rate = train_df['is_fraud'].mean()
print("=" * 60)
print("TARGET DISTRIBUTION")
print("=" * 60)
print(f"Fraud rate: {fraud_rate:.4f} ({fraud_rate*100:.2f}%)")
print(f"Non-fraud: {(1-fraud_rate)*100:.2f}%")
print(f"\nClass counts:")
print(train_df['is_fraud'].value_counts())

# Check missing values
print("\n" + "=" * 60)
print("MISSING VALUES")
print("=" * 60)
missing_train = train_df.isnull().sum()
missing_test = test_df.isnull().sum()
print("\nTraining data missing values:")
print(missing_train[missing_train > 0])
print(f"\nTotal missing in train: {missing_train.sum()}")
print(f"Total missing in test: {missing_test.sum()}")

# Data types
print("\n" + "=" * 60)
print("DATA TYPES")
print("=" * 60)
print(train_df.dtypes)

TARGET DISTRIBUTION
Fraud rate: 0.1413 (14.13%)
Non-fraud: 85.87%

Class counts:
is_fraud
0    85872
1    14128
Name: count, dtype: int64

MISSING VALUES

Training data missing values:
transaction_amount        2419
avg_transaction_amount    2419
device_trust_score        2419
dtype: int64

Total missing in train: 7257
Total missing in test: 7212

DATA TYPES
ID                          int64
transaction_id             object
user_id                     int64
age                         int64
gender                     object
country                    object
device_type                object
device_os                  object
merchant_category          object
transaction_amount        float64
transaction_type           object
time_of_day                 int64
day_of_week                 int64
transaction_duration      float64
num_prev_transactions       int64
avg_transaction_amount    float64
std_transaction_amount    float64
transactions_last_24h       int64
transactions_last_1h       

---

# 4. Preprocessing Pipeline

The preprocessing pipeline is implemented sequentially (without functions) for clarity.

## 4.1 Separate Features and Target

Extract the target variable and prepare feature matrices.

In [17]:
# Store test IDs for submission
test_ids = test_df['ID'].values

# Extract target variable
y_train = train_df['is_fraud'].values

# Columns to drop (IDs are not useful as features)
cols_to_drop = ['ID', 'transaction_id', 'user_id']

# Create feature DataFrames
train_features = train_df.drop(columns=['is_fraud'] + cols_to_drop, errors='ignore')
test_features = test_df.drop(columns=cols_to_drop, errors='ignore')

# Identify column types
numerical_cols = train_features.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train_features.select_dtypes(include=['object', 'category']).columns.tolist()

print("=" * 60)
print("FEATURES SEPARATED")
print("=" * 60)
print(f"\nNumerical columns ({len(numerical_cols)}):")
print(numerical_cols)
print(f"\nCategorical columns ({len(categorical_cols)}):")
print(categorical_cols)
print(f"\nTraining features shape: {train_features.shape}")
print(f"Test features shape: {test_features.shape}")

FEATURES SEPARATED

Numerical columns (21):
['age', 'transaction_amount', 'time_of_day', 'day_of_week', 'transaction_duration', 'num_prev_transactions', 'avg_transaction_amount', 'std_transaction_amount', 'transactions_last_24h', 'transactions_last_1h', 'failed_login_attempts', 'ip_risk_score', 'device_trust_score', 'account_age_days', 'has_chargeback_history', 'shared_ip_users', 'shared_device_users', 'merchant_risk', 'country_risk', 'distance_from_home', 'is_new_country']

Categorical columns (6):
['gender', 'country', 'device_type', 'device_os', 'merchant_category', 'transaction_type']

Training features shape: (100000, 27)
Test features shape: (100000, 27)


## 4.2 Handle Missing Values with KNN Imputation

**KNN Imputation** uses the k-nearest neighbors algorithm to impute missing values based on the similarity of other features. This is more sophisticated than simple mean/mode imputation as it considers the local structure of the data.

- **Numerical columns**: KNN imputation with distance-weighted neighbors (k=5)
- **Categorical columns**: Mode imputation (most frequent value)

In [18]:
# ============================================================================
# KNN IMPUTATION FOR NUMERICAL COLUMNS
# ============================================================================

KNN_NEIGHBORS = 5  # Number of neighbors for KNN imputation
imputation_info = {}  # Store imputation info for test data

# Get numerical columns that exist in both train and test
numerical_cols_in_df = [col for col in numerical_cols if col in train_features.columns]

# Check if there are any missing values in numerical columns
has_missing_numerical = train_features[numerical_cols_in_df].isnull().any().any()

if has_missing_numerical:
    print(f"Applying KNN imputation (k={KNN_NEIGHBORS}) for numerical columns...")
    
    # Create and fit KNN imputer
    knn_imputer = KNNImputer(n_neighbors=KNN_NEIGHBORS, weights='distance')
    
    # Fit on training data and transform
    numerical_data_train = train_features[numerical_cols_in_df].values
    imputed_train = knn_imputer.fit_transform(numerical_data_train)
    
    # Update training DataFrame
    train_features[numerical_cols_in_df] = imputed_train
    
    # Store imputer for test data
    imputation_info['knn_imputer'] = knn_imputer
    imputation_info['numerical_cols'] = numerical_cols_in_df
    
    print(f"✓ Imputed {train_df[numerical_cols_in_df].isnull().sum().sum()} missing values in training data")
else:
    print("No missing values in numerical columns")
    imputation_info['knn_imputer'] = None
    imputation_info['numerical_cols'] = numerical_cols_in_df

# ============================================================================
# MODE IMPUTATION FOR CATEGORICAL COLUMNS
# ============================================================================

print("\nApplying mode imputation for categorical columns...")
mode_values = {}

for col in categorical_cols:
    if col in train_features.columns and train_features[col].isnull().any():
        # Get the mode (most frequent value)
        mode_value = train_features[col].mode()
        if len(mode_value) > 0:
            mode_value = mode_value[0]
            train_features[col] = train_features[col].fillna(mode_value)
            mode_values[col] = mode_value
            print(f"  ✓ {col}: filled with '{mode_value}'")

imputation_info['mode_values'] = mode_values

print("\n✓ Missing value handling complete for training data")

Applying KNN imputation (k=5) for numerical columns...
✓ Imputed 7257 missing values in training data

Applying mode imputation for categorical columns...

✓ Missing value handling complete for training data


In [19]:
# ============================================================================
# APPLY IMPUTATION TO TEST DATA
# ============================================================================

print("Applying imputation to test data...")

# Apply KNN imputation for numerical columns
if imputation_info['knn_imputer'] is not None:
    knn_imputer = imputation_info['knn_imputer']
    numerical_cols_imp = imputation_info['numerical_cols']
    
    cols_to_impute = [col for col in numerical_cols_imp if col in test_features.columns]
    
    if len(cols_to_impute) > 0 and test_features[cols_to_impute].isnull().any().any():
        numerical_data_test = test_features[cols_to_impute].values
        imputed_test = knn_imputer.transform(numerical_data_test)
        test_features[cols_to_impute] = imputed_test
        print(f"✓ KNN imputation applied to test numerical columns")

# Apply mode imputation for categorical columns
for col, mode_val in imputation_info['mode_values'].items():
    if col in test_features.columns and test_features[col].isnull().any():
        test_features[col] = test_features[col].fillna(mode_val)
        print(f"✓ Mode imputation applied to test column: {col}")

# Handle any remaining missing values in test
for col in numerical_cols:
    if col in test_features.columns and test_features[col].isnull().any():
        test_features[col] = test_features[col].fillna(0)
        
for col in categorical_cols:
    if col in test_features.columns and test_features[col].isnull().any():
        mode_val = train_features[col].mode()[0] if len(train_features[col].mode()) > 0 else 'unknown'
        test_features[col] = test_features[col].fillna(mode_val)

print("\n✓ Imputation complete for both train and test data")
print(f"Remaining missing in train: {train_features.isnull().sum().sum()}")
print(f"Remaining missing in test: {test_features.isnull().sum().sum()}")

Applying imputation to test data...
✓ KNN imputation applied to test numerical columns

✓ Imputation complete for both train and test data
Remaining missing in train: 0
Remaining missing in test: 0


## 4.3 Feature Engineering: Ratio Features

Create meaningful ratio features that capture relationships between existing features. These ratios can help detect anomalous transaction patterns.

In [20]:
# ============================================================================
# CREATE RATIO FEATURES
# ============================================================================

print("Creating ratio features...")

# Define meaningful ratio pairs for fraud detection
# (numerator, denominator, new_feature_name)
ratio_pairs = [
    ('transaction_amount', 'avg_transaction_amount', 'amount_vs_avg_ratio'),
    ('transaction_amount', 'std_transaction_amount', 'amount_vs_std_ratio'),
    ('transactions_last_1h', 'transactions_last_24h', 'hourly_vs_daily_ratio'),
    ('failed_login_attempts', 'num_prev_transactions', 'failed_vs_total_ratio'),
    ('shared_ip_users', 'shared_device_users', 'ip_vs_device_shared_ratio'),
]

for num, denom, name in ratio_pairs:
    if num in train_features.columns and denom in train_features.columns:
        # Avoid division by zero by replacing 0 with small value
        denom_safe_train = train_features[denom].replace(0, 1e-10)
        denom_safe_test = test_features[denom].replace(0, 1e-10)
        
        train_features[name] = train_features[num] / denom_safe_train
        test_features[name] = test_features[num] / denom_safe_test
        print(f"  ✓ Created: {name}")

print(f"\n✓ Ratio features created")
print(f"Current train shape: {train_features.shape}")

Creating ratio features...
  ✓ Created: amount_vs_avg_ratio
  ✓ Created: amount_vs_std_ratio
  ✓ Created: hourly_vs_daily_ratio
  ✓ Created: failed_vs_total_ratio
  ✓ Created: ip_vs_device_shared_ratio

✓ Ratio features created
Current train shape: (100000, 32)


## 4.4 Feature Engineering: Log-Transformed Features

Apply log transformation to skewed numerical features. This helps normalize distributions and can improve model performance.

In [21]:
# ============================================================================
# CREATE LOG-TRANSFORMED FEATURES
# ============================================================================

print("Creating log-transformed features...")

# Columns that typically benefit from log transformation (skewed distributions)
log_candidates = [
    'transaction_amount', 
    'avg_transaction_amount', 
    'std_transaction_amount',
    'account_age_days', 
    'distance_from_home', 
    'num_prev_transactions'
]

for col in log_candidates:
    if col in train_features.columns:
        # Check if column has non-negative values
        min_val_train = train_features[col].min()
        min_val_test = test_features[col].min()
        
        if min_val_train >= 0 and min_val_test >= 0:
            # Use log1p (log(1+x)) for numerical stability
            train_features[f'{col}_log'] = np.log1p(train_features[col])
            test_features[f'{col}_log'] = np.log1p(test_features[col])
            print(f"  ✓ Created: {col}_log")

print(f"\n✓ Log features created")
print(f"Current train shape: {train_features.shape}")

Creating log-transformed features...
  ✓ Created: transaction_amount_log
  ✓ Created: std_transaction_amount_log
  ✓ Created: account_age_days_log
  ✓ Created: distance_from_home_log
  ✓ Created: num_prev_transactions_log

✓ Log features created
Current train shape: (100000, 37)


## 4.5 Feature Engineering: Interaction Features

Create domain-specific interaction features that capture complex patterns in fraud behavior. These features combine multiple signals to create more informative predictors.

In [22]:
# ============================================================================
# CREATE INTERACTION FEATURES
# ============================================================================

print("Creating interaction features...")

# --- Risk Score Interactions ---
if 'ip_risk_score' in train_features.columns and 'device_trust_score' in train_features.columns:
    # Combined risk indicator (high IP risk + low device trust)
    train_features['risk_interaction'] = train_features['ip_risk_score'] * (1 - train_features['device_trust_score'] / 100)
    test_features['risk_interaction'] = test_features['ip_risk_score'] * (1 - test_features['device_trust_score'] / 100)
    print("  ✓ risk_interaction")

if 'merchant_risk' in train_features.columns and 'country_risk' in train_features.columns:
    # Combined merchant-country risk
    train_features['merchant_country_risk'] = train_features['merchant_risk'] * train_features['country_risk']
    test_features['merchant_country_risk'] = test_features['merchant_risk'] * test_features['country_risk']
    print("  ✓ merchant_country_risk")

# --- Transaction Amount Anomaly ---
if 'transaction_amount' in train_features.columns and 'avg_transaction_amount' in train_features.columns:
    if 'std_transaction_amount' in train_features.columns:
        # Z-score like feature
        train_features['amount_zscore'] = (train_features['transaction_amount'] - train_features['avg_transaction_amount']) / (train_features['std_transaction_amount'] + 1e-6)
        test_features['amount_zscore'] = (test_features['transaction_amount'] - test_features['avg_transaction_amount']) / (test_features['std_transaction_amount'] + 1e-6)
        print("  ✓ amount_zscore")

# --- Velocity-based Features ---
if 'transactions_last_24h' in train_features.columns and 'transactions_last_1h' in train_features.columns:
    # Recent activity concentration
    train_features['hourly_concentration'] = train_features['transactions_last_1h'] / (train_features['transactions_last_24h'] + 1)
    test_features['hourly_concentration'] = test_features['transactions_last_1h'] / (test_features['transactions_last_24h'] + 1)
    print("  ✓ hourly_concentration")

# --- Account Trust Features ---
if 'account_age_days' in train_features.columns and 'num_prev_transactions' in train_features.columns:
    # Transaction frequency per day of account age
    train_features['tx_per_day_age'] = train_features['num_prev_transactions'] / (train_features['account_age_days'] + 1)
    test_features['tx_per_day_age'] = test_features['num_prev_transactions'] / (test_features['account_age_days'] + 1)
    print("  ✓ tx_per_day_age")

# --- Location Risk ---
if 'is_new_country' in train_features.columns and 'distance_from_home' in train_features.columns:
    train_features['new_location_distance'] = train_features['is_new_country'] * train_features['distance_from_home']
    test_features['new_location_distance'] = test_features['is_new_country'] * test_features['distance_from_home']
    print("  ✓ new_location_distance")

# --- Failed Login Impact ---
if 'failed_login_attempts' in train_features.columns and 'transaction_amount' in train_features.columns:
    train_features['failed_login_amount'] = train_features['failed_login_attempts'] * train_features['transaction_amount']
    test_features['failed_login_amount'] = test_features['failed_login_attempts'] * test_features['transaction_amount']
    print("  ✓ failed_login_amount")

# --- Shared Resource Risk ---
if 'shared_ip_users' in train_features.columns and 'shared_device_users' in train_features.columns:
    train_features['total_shared_users'] = train_features['shared_ip_users'] + train_features['shared_device_users']
    test_features['total_shared_users'] = test_features['shared_ip_users'] + test_features['shared_device_users']
    
    train_features['shared_resource_product'] = train_features['shared_ip_users'] * train_features['shared_device_users']
    test_features['shared_resource_product'] = test_features['shared_ip_users'] * test_features['shared_device_users']
    print("  ✓ total_shared_users, shared_resource_product")

# --- Time-based Risk ---
if 'time_of_day' in train_features.columns:
    # Night time risk (0-6 AM and 22-24)
    train_features['is_night_time'] = ((train_features['time_of_day'] >= 0) & (train_features['time_of_day'] <= 6) | 
                                       (train_features['time_of_day'] >= 22)).astype(float)
    test_features['is_night_time'] = ((test_features['time_of_day'] >= 0) & (test_features['time_of_day'] <= 6) | 
                                      (test_features['time_of_day'] >= 22)).astype(float)
    print("  ✓ is_night_time")

if 'day_of_week' in train_features.columns:
    # Weekend indicator
    train_features['is_weekend'] = (train_features['day_of_week'] >= 5).astype(float)
    test_features['is_weekend'] = (test_features['day_of_week'] >= 5).astype(float)
    print("  ✓ is_weekend")

# --- Chargeback History Interaction ---
if 'has_chargeback_history' in train_features.columns and 'transaction_amount' in train_features.columns:
    median_amount = train_features['transaction_amount'].median()
    train_features['chargeback_high_amount'] = train_features['has_chargeback_history'] * (train_features['transaction_amount'] > median_amount).astype(float)
    test_features['chargeback_high_amount'] = test_features['has_chargeback_history'] * (test_features['transaction_amount'] > median_amount).astype(float)
    print("  ✓ chargeback_high_amount")

# --- Additional High-Value Interactions ---
if 'ip_risk_score' in train_features.columns and 'transaction_amount' in train_features.columns:
    train_features['high_risk_high_amount'] = train_features['ip_risk_score'] * train_features['transaction_amount']
    test_features['high_risk_high_amount'] = test_features['ip_risk_score'] * test_features['transaction_amount']
    print("  ✓ high_risk_high_amount")

if 'failed_login_attempts' in train_features.columns and 'ip_risk_score' in train_features.columns:
    train_features['failed_login_risk'] = train_features['failed_login_attempts'] * train_features['ip_risk_score']
    test_features['failed_login_risk'] = test_features['failed_login_attempts'] * test_features['ip_risk_score']
    print("  ✓ failed_login_risk")

if 'is_new_country' in train_features.columns and 'transaction_amount' in train_features.columns:
    train_features['new_country_amount'] = train_features['is_new_country'] * train_features['transaction_amount']
    test_features['new_country_amount'] = test_features['is_new_country'] * test_features['transaction_amount']
    print("  ✓ new_country_amount")

# --- Squared Features for Important Risk Indicators ---
for col in ['ip_risk_score', 'merchant_risk', 'country_risk']:
    if col in train_features.columns:
        train_features[f'{col}_squared'] = train_features[col] ** 2
        test_features[f'{col}_squared'] = test_features[col] ** 2
        print(f"  ✓ {col}_squared")

# --- Combined Risk Score ---
risk_cols = []
for col in ['ip_risk_score', 'merchant_risk', 'country_risk']:
    if col in train_features.columns:
        risk_cols.append(col)

if len(risk_cols) > 0:
    train_features['combined_risk'] = train_features[risk_cols].mean(axis=1)
    test_features['combined_risk'] = test_features[risk_cols].mean(axis=1)
    print("  ✓ combined_risk")

print(f"\n✓ Interaction features created")
print(f"Current train shape: {train_features.shape}")

Creating interaction features...
  ✓ risk_interaction
  ✓ merchant_country_risk
  ✓ amount_zscore
  ✓ hourly_concentration
  ✓ tx_per_day_age
  ✓ new_location_distance
  ✓ failed_login_amount
  ✓ total_shared_users, shared_resource_product
  ✓ is_night_time
  ✓ is_weekend
  ✓ chargeback_high_amount
  ✓ high_risk_high_amount
  ✓ failed_login_risk
  ✓ new_country_amount
  ✓ ip_risk_score_squared
  ✓ merchant_risk_squared
  ✓ country_risk_squared
  ✓ combined_risk

✓ Interaction features created
Current train shape: (100000, 56)


## 4.6 One-Hot Encode Categorical Features

Convert categorical variables into binary dummy variables using one-hot encoding.

In [23]:
# ============================================================================
# ONE-HOT ENCODING
# ============================================================================

print("Applying one-hot encoding to categorical features...")

encoding_info = {}

for col in categorical_cols:
    if col in train_features.columns:
        # Get unique categories from training data (sorted for consistency)
        categories = train_features[col].unique()
        categories = [c for c in categories if pd.notna(c)]
        categories = sorted(categories)
        
        encoding_info[col] = categories
        
        # Create dummy columns for each category
        for category in categories:
            new_col_name = f"{col}_{category}"
            train_features[new_col_name] = (train_features[col] == category).astype(int)
            test_features[new_col_name] = (test_features[col] == category).astype(int)
        
        # Drop original categorical column
        train_features = train_features.drop(columns=[col])
        test_features = test_features.drop(columns=[col])
        
        print(f"  ✓ {col}: {len(categories)} categories encoded")

print(f"\n✓ One-hot encoding complete")
print(f"Train shape after encoding: {train_features.shape}")
print(f"Test shape after encoding: {test_features.shape}")

Applying one-hot encoding to categorical features...
  ✓ gender: 2 categories encoded
  ✓ country: 10 categories encoded
  ✓ device_type: 3 categories encoded
  ✓ device_os: 5 categories encoded
  ✓ merchant_category: 9 categories encoded
  ✓ transaction_type: 4 categories encoded

✓ One-hot encoding complete
Train shape after encoding: (100000, 83)
Test shape after encoding: (100000, 83)


## 4.7 Align Train and Test Columns

Ensure both datasets have the same columns after all transformations.

In [24]:
# ============================================================================
# ALIGN COLUMNS BETWEEN TRAIN AND TEST
# ============================================================================

print("Aligning train and test columns...")

train_cols = set(train_features.columns)
test_cols = set(test_features.columns)

# Add missing columns to test (with zeros)
missing_in_test = train_cols - test_cols
for col in missing_in_test:
    test_features[col] = 0
if missing_in_test:
    print(f"  Added {len(missing_in_test)} missing columns to test data")

# Add missing columns to train (shouldn't happen often)
missing_in_train = test_cols - train_cols
for col in missing_in_train:
    train_features[col] = 0
if missing_in_train:
    print(f"  Added {len(missing_in_train)} missing columns to train data")

# Ensure same column order (sorted)
all_cols = sorted(train_features.columns.tolist())
train_features = train_features[all_cols]
test_features = test_features[all_cols]

print(f"\n✓ Columns aligned")
print(f"Final number of features: {len(all_cols)}")

Aligning train and test columns...

✓ Columns aligned
Final number of features: 83


## 4.8 Clip Outliers

Apply percentile-based outlier clipping (winsorization) to limit extreme values.

In [25]:
# ============================================================================
# CLIP OUTLIERS (WINSORIZATION)
# ============================================================================

print("Clipping outliers (1st to 99th percentile)...")

train_values = train_features.values

def clip_outliers(X, lower_percentile=1, upper_percentile=99):
    X = X.copy()
    n_features = X.shape[1]
    clip_bounds = {'lower': [], 'upper': []}
    for i in range(n_features):
        lower = np.percentile(X[:, i], lower_percentile)
        upper = np.percentile(X[:, i], upper_percentile)
        clip_bounds['lower'].append(lower)
        clip_bounds['upper'].append(upper)
        X[:, i] = np.clip(X[:, i], lower, upper)
    return X, clip_bounds

def apply_clip_outliers(X, clip_bounds):
    X = X.copy()
    for i in range(X.shape[1]):
        X[:, i] = np.clip(X[:, i], clip_bounds['lower'][i], clip_bounds['upper'][i])
    return X

train_clipped, clip_bounds = clip_outliers(train_values, lower_percentile=1, upper_percentile=99)
test_clipped = apply_clip_outliers(test_features.values, clip_bounds)

print(f"\n✓ Outlier clipping complete")
print(f"Train shape after clipping: {train_clipped.shape}")
print(f"Test shape after clipping: {test_clipped.shape}")

Clipping outliers (1st to 99th percentile)...

✓ Outlier clipping complete
Train shape after clipping: (100000, 83)
Test shape after clipping: (100000, 83)


## 4.9 Standardize Features

Standardize features to zero mean and unit variance using a custom StandardScaler (from scratch).

In [26]:
# Standardize features (zero mean, unit variance) without using sklearn
# Fit scaler on train, apply to both train and test

means = X_train.mean(axis=0)
stds = X_train.std(axis=0)

# Avoid division by zero
stds_replaced = stds.replace(0, 1)

X_train_scaled = (X_train - means) / stds_replaced
X_test_scaled = (X_test - means) / stds_replaced

print("Standardization complete.")
print(f"Train shape: {X_train_scaled.shape}, Test shape: {X_test_scaled.shape}")

NameError: name 'X_train' is not defined

## 5. Train/Validation Split

Split the training data into train and validation sets for model evaluation.

In [ ]:
# Split train data into train/validation sets (80/20 split, stratified)
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_scaled, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"Train set: {X_tr.shape}, Validation set: {X_val.shape}")

## 6. Metrics and Helper Functions

Define evaluation metrics (AUC, accuracy, etc.) and helper functions for model evaluation.

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def predict_proba(X, w, b):
    return sigmoid(np.dot(X, w) + b)

def predict(X, w, b, threshold=0.5):
    return (predict_proba(X, w, b) >= threshold).astype(int)

def evaluate_metrics(y_true, y_pred_proba, y_pred):
    auc = roc_auc_score(y_true, y_pred_proba)
    acc = accuracy_score(y_true, y_pred)
    print(f"AUC: {auc:.4f}, Accuracy: {acc:.4f}")
    return auc, acc

print("Metrics and helper functions defined.")

## 7. Logistic Regression Model (from Scratch)

Define the custom Logistic Regression class with Adam optimizer, elastic net regularization, and early stopping.

In [ ]:
class LogisticRegressionScratch:
    def __init__(self, lr=0.01, n_iter=1000, batch_size=64, l1=0.0, l2=0.0, beta1=0.9, beta2=0.999, eps=1e-8, early_stopping_rounds=20, verbose=True):
        self.lr = lr
        self.n_iter = n_iter
        self.batch_size = batch_size
        self.l1 = l1
        self.l2 = l2
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.early_stopping_rounds = early_stopping_rounds
        self.verbose = verbose
        self.w = None
        self.b = None
        self.loss_history = []
        self.val_loss_history = []
        self.best_weights = None
        self.best_val_loss = float('inf')
        self.no_improve_rounds = 0

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def _loss(self, y, y_pred):
        # Binary cross-entropy + elastic net
        eps = 1e-8
        ce = -np.mean(y * np.log(y_pred + eps) + (1 - y) * np.log(1 - y_pred + eps))
        l1_penalty = self.l1 * np.sum(np.abs(self.w))
        l2_penalty = self.l2 * np.sum(self.w ** 2)
        return ce + l1_penalty + l2_penalty

    def fit(self, X, y, X_val=None, y_val=None):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0.0
        m_w, v_w = np.zeros(n_features), np.zeros(n_features)
        m_b, v_b = 0.0, 0.0
        t = 0
        for epoch in range(self.n_iter):
            # Shuffle
            idx = np.random.permutation(n_samples)
            X_shuf, y_shuf = X[idx], y[idx]
            for start in range(0, n_samples, self.batch_size):
                end = min(start + self.batch_size, n_samples)
                X_batch, y_batch = X_shuf[start:end], y_shuf[start:end]
                y_pred = self._sigmoid(np.dot(X_batch, self.w) + self.b)
                error = y_pred - y_batch
                grad_w = np.dot(X_batch.T, error) / len(y_batch) + self.l1 * np.sign(self.w) + 2 * self.l2 * self.w
                grad_b = np.mean(error)
                t += 1
                # Adam update
                m_w = self.beta1 * m_w + (1 - self.beta1) * grad_w
                v_w = self.beta2 * v_w + (1 - self.beta2) * (grad_w ** 2)
                m_w_hat = m_w / (1 - self.beta1 ** t)
                v_w_hat = v_w / (1 - self.beta2 ** t)
                self.w -= self.lr * m_w_hat / (np.sqrt(v_w_hat) + self.eps)
                m_b = self.beta1 * m_b + (1 - self.beta1) * grad_b
                v_b = self.beta2 * v_b + (1 - self.beta2) * (grad_b ** 2)
                m_b_hat = m_b / (1 - self.beta1 ** t)
                v_b_hat = v_b / (1 - self.beta2 ** t)
                self.b -= self.lr * m_b_hat / (np.sqrt(v_b_hat) + self.eps)
            # Track loss
            y_pred_train = self._sigmoid(np.dot(X, self.w) + self.b)
            train_loss = self._loss(y, y_pred_train)
            self.loss_history.append(train_loss)
            if X_val is not None and y_val is not None:
                y_pred_val = self._sigmoid(np.dot(X_val, self.w) + self.b)
                val_loss = self._loss(y_val, y_pred_val)
                self.val_loss_history.append(val_loss)
                if self.verbose:
                    print(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}")
                # Early stopping
                if val_loss < self.best_val_loss - 1e-5:
                    self.best_val_loss = val_loss
                    self.best_weights = (self.w.copy(), self.b)
                    self.no_improve_rounds = 0
                else:
                    self.no_improve_rounds += 1
                if self.no_improve_rounds >= self.early_stopping_rounds:
                    if self.verbose:
                        print(f"Early stopping at epoch {epoch+1}")
                    if self.best_weights is not None:
                        self.w, self.b = self.best_weights
                    return
            else:
                if self.verbose:
                    print(f"Epoch {epoch+1}: train_loss={train_loss:.4f}")
        # Restore best weights if early stopping
        if self.best_weights is not None:
            self.w, self.b = self.best_weights

    def predict_proba(self, X):
        return self._sigmoid(np.dot(X, self.w) + self.b)

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)

print("Custom Logistic Regression class defined.")

## 8. Model Training

Train the custom logistic regression model on the training set and monitor validation loss for early stopping.

In [ ]:
# Instantiate and train the model
model = LogisticRegressionScratch(
    lr=0.01, n_iter=200, batch_size=128, l1=1e-4, l2=1e-4, early_stopping_rounds=10, verbose=True
)
model.fit(X_tr.values, y_tr.values, X_val.values, y_val.values)

print("Model training complete.")

## 9. Validation Performance and Threshold Tuning

Evaluate the model on the validation set, tune the threshold for best AUC, and print metrics.

In [ ]:
# Evaluate on validation set and tune threshold for best AUC
val_proba = model.predict_proba(X_val.values)

from sklearn.metrics import roc_auc_score
best_auc = 0
best_threshold = 0.5
for thresh in np.arange(0.1, 0.9, 0.01):
    val_pred = (val_proba >= thresh).astype(int)
    auc = roc_auc_score(y_val, val_proba)
    if auc > best_auc:
        best_auc = auc
        best_threshold = thresh

val_pred = (val_proba >= best_threshold).astype(int)
auc, acc = evaluate_metrics(y_val.values, val_proba, val_pred)
print(f"Best threshold: {best_threshold:.2f}")

## 10. Predict on Test Set and Prepare Submission

Use the trained model to predict probabilities on the test set, apply the best threshold, and prepare the submission file.

In [ ]:
# Predict on test set
proba_test = model.predict_proba(X_test_scaled.values)
pred_test = (proba_test >= best_threshold).astype(int)

# Prepare submission
submission = pd.DataFrame({
    'id': test['id'],
    'label': pred_test
})
submission.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' created.")

## 11. Save Model and Scaler Parameters

Save the trained model weights and scaler parameters for reproducibility.

In [ ]:
import json

# Save model weights
model_params = {
    'weights': model.w.tolist(),
    'bias': float(model.b)
}
with open('logistic_model.json', 'w') as f:
    json.dump(model_params, f)

# Save scaler parameters
scaler_params = {
    'means': means.tolist(),
    'stds': stds.tolist()
}
with open('scaler_params.json', 'w') as f:
    json.dump(scaler_params, f)

print("Model and scaler parameters saved.")

---

# Pipeline Complete

This notebook provides a fully transparent, step-by-step implementation of the fraud detection pipeline, with all preprocessing visible and only the model code encapsulated in a class. You can now run all cells to reproduce the results and generate a submission.

In [ ]:
# ============================================================================
# CLIP OUTLIERS (WINSORIZATION)
# ============================================================================

print("Clipping outliers using percentile-based bounds...")

LOWER_PERCENTILE = 1
UPPER_PERCENTILE = 99

# Convert to numpy arrays
train_values = train_features.values.copy()
test_values = test_features.values.copy()

n_features = train_values.shape[1]
clip_bounds = {'lower': [], 'upper': []}

# Compute clip bounds from training data and apply to both
for i in range(n_features):
    lower = np.percentile(train_values[:, i], LOWER_PERCENTILE)
    upper = np.percentile(train_values[:, i], UPPER_PERCENTILE)
    
    clip_bounds['lower'].append(lower)
    clip_bounds['upper'].append(upper)
    
    # Clip training data
    train_values[:, i] = np.clip(train_values[:, i], lower, upper)
    
    # Clip test data using same bounds
    test_values[:, i] = np.clip(test_values[:, i], lower, upper)

print(f"✓ Outliers clipped to [{LOWER_PERCENTILE}th, {UPPER_PERCENTILE}th] percentile")